#Εργαστηριακό Project #2: Εκτίμηση Οπτικής Ροής (Optical Flow) και Εξαγωγή Χαρακτηριστικών σε Βίντεο για Αναγνώριση Δράσεων

##Imports

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.io
from scipy.stats import multivariate_normal
from scipy.ndimage import map_coordinates, label
import sys
from matplotlib.patches import Circle
import timeit
import time
import random
from sklearn.cluster import KMeans
from scipy.spatial import distance_matrix
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cdist
from numpy.linalg import norm
#import cv21_lab2_2_utils as utils
from scipy.ndimage import map_coordinates
import matplotlib.patches as patches
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image 

%matplotlib inline

##Μέρος 1: Παρακολούθηση Προσώπου και Χεριών με Χρήση της Μεθόδου Οπτικής Ροής των Lucas-Kanade

###1.1 Ανίχνευση Δέρματος Προσώπου και Χεριών

In [ ]:
mat_file = scipy.io.loadmat("skinSamplesRGB.mat")
#RGB to YCbCr 
skin_sam_RGB = mat_file['skinSamplesRGB'] # array that contains R, G, B arrays. Each one of the three arrays has 22 lines and 81 columns.
#R,G,B = cv2.split(skin_sam_RGB)

skin_sam_YCbCr = cv2.cvtColor(skin_sam_RGB, cv2.COLOR_RGB2YCrCb)
#_, Cb1, Cr1 = cv2.split(skin_sam_YCbCr) #same outputs as below
Cr2, Cb2 = skin_sam_YCbCr[:,:,2], skin_sam_YCbCr[:,:,1] 
samples = Cr2.shape[0]*Cr2.shape[1]
mean = np.array([np.sum(Cb2)/samples, np.sum(Cr2)/samples])

#c = np.array([Cb2, Cr2]).transpose()
cov = np.array(np.cov(Cb2.reshape(1,-1)[0], Cr2.reshape(1,-1)[0]))


In [ ]:
def face_detect(I, mean, cov):
        
    Ι = cv2.cvtColor(I, cv2.COLOR_RGB2YCrCb)
    
    #_, Cb1, Cr1 = cv2.split(skin_sam_YCbCr) same outputs as below
    Cr, Cb = Ι[:,:,2], Ι[:,:,1]
    c = np.array([Cb, Cr]).transpose()
    
    P = multivariate_normal.pdf(c, list(mean), cov)
    _,Pthres = cv2.threshold(P, 0.0005, 0.25, cv2.THRESH_BINARY)
    Pthres = Pthres.transpose()
    
    #opening with small structural
    kernel_opening = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(Pthres, cv2.MORPH_OPEN, kernel_opening)
    
    
    #closing with big structural
    kernel_closing = np.ones((18,18),np.uint8)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_closing)
    
    labeled_array, num_features = label(closing)
    
    f, arr = plt.subplots(2,2, figsize=(9, 9))
    arr[0,0].imshow(I)
    arr[0,0].title.set_text('Initial_Image')
    arr[0,1].imshow(Pthres)
    arr[0,1].title.set_text('Thresholded_Image')
    arr[1,0].imshow(opening)
    arr[1,0].title.set_text('Opened_Image')
    arr[1,1].imshow(closing)
    arr[1,1].title.set_text('Closed_Image')
    
    bounding_boxes = []
    for i in range(1, num_features+1):
        x,y = np.where(labeled_array == i)
        x = np.sort(x) 
        y = np.sort(y) 
        width = x[-1]-x[0]
        height = y[-1] - y[0]
        bounding_boxes.append([y[0], x[0],height,width])
    
    return bounding_boxes

In [ ]:
img = cv2.cvtColor(cv2.imread("1.png"), cv2.COLOR_BGR2RGB)
collect_array = face_detect(img, mean, cov)
#print(mean,cov)

In [ ]:
image = Image.open('1.png')

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(image)

for i in range(len(collect_array)):
    # Create a Rectangle patch
    rect = patches.Rectangle((collect_array[i][0], collect_array[i][1]), collect_array[i][2], collect_array[i][3], linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)


plt.title('Bounding_Boxes')
plt.show(fig)

###1.2 Παρακολούθηση Προσώπου και Χεριών

In [ ]:
def Gauss(img,sigma):
    n = int(2*np.ceil(3*sigma)+1)
    gauss1D = cv2.getGaussianKernel(n, sigma) # Column vector
    gauss2D = gauss1D @ gauss1D.T # Symmetric gaussian kernel
    return cv2.filter2D(img, -1, gauss2D)

def disk_strel(n):
    '''
        Return a structural element, which is a disk of radius n.
    '''
    r = int(np.round(n))
    d = 2*r+1
    x = np.arange(d) - r
    y = np.arange(d) - r
    x, y = np.meshgrid(x,y)
    strel = x**2 + y**2 <= r**2
    return strel.astype(np.uint8)

def imageread(filename):
  img = cv2.imread(filename)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  # Normalize to [0,1]
  img = img.astype(np.float)/255
  # Show image
  return img
  
img = imageread("1.png")
plt.title('FACE[138; 88; 73; 123]')
plt.imshow(img[88:123+88,138:138+73])
plt.show()

plt.title('LEFT_HAND[47, 243, 71, 66]')
plt.imshow(img[264:264+48,162:162+83])
plt.show()

plt.title('RIGHT_HAND[162, 264, 83, 48]')
plt.imshow(img[243:243+66,47:47+71])
plt.show()

####1.2.1 Υλοποίηση του Αλγόριθμου των Lucas-Kanade

In [ ]:
def Lucas_Kanade(img1,img2,rho,epsilon,d_x0, d_y0): 

    (rows,cols) = img1.shape
    [x_0,y_0] = np.meshgrid(np.arange(0,cols),np.arange(0,rows))

    dx = d_x0
    dy = d_y0

    [Iy, Ix] = np.gradient(img1)

    for i in range(10):
        I1_di = map_coordinates(img1,[y_0+dy,x_0+dx],order=1, mode='constant')

        E = img2 - I1_di#.reshape((rows,cols))

        A1 = map_coordinates(Ix, [y_0+dy,x_0+dx],order=1, mode='constant')#.reshape((rows,cols))
        A2 = map_coordinates(Iy, [y_0+dy,x_0+dx],order=1, mode='constant')#.reshape((rows,cols))
        A1A2 = A1*A2
        a = Gauss(A1*A1, rho) + epsilon
        b = Gauss(A1A2, rho)
        c = Gauss(A1A2, rho)
        d = Gauss(A2*A2, rho) + epsilon
        e = Gauss(A1*E, rho)
        f = Gauss(A2*E, rho)     

        det = (a*d - b*c)
        u = [(d*e - b*f)/det , (-c*e + a*f)/det]
        dy = dy + u[1]
        dx = dx + u[0]
        

    return (dx, dy)

####1.2.2 Υπολογισμός της Μετατόπισης των Παραθύρων από τα Διανύσματα Οπτικής Ροής

In [ ]:
def displ(d_x, d_y, threshold):
        dx = d_x.flatten()
        dy = d_y.flatten()
        d_sq = dx*dx + dy*dy
        
        condition = (d_sq >= threshold)
        
        displ_x, displ_y = 0,0
        count = 0
        
        for i in range(len(d_sq)):
            if condition[i]:
                displ_x -= dx[i]
                displ_y -= dy[i]
                count += 1
        if count==0:
            return (0,0)
#         print("count = ", count," x + ",displ_x/count," y + ", displ_y/count)
        return displ_x/count, displ_y/count

In [ ]:
start_img = cv2.cvtColor(cv2.imread("1.png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
next_img = cv2.cvtColor(cv2.imread("2.png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255

#right hand
x, y, width, height = 47, 243, 71, 66
#left hand
# x, y, width, height = 162, 264, 83, 48
# # face
#x, y, width, height = 138, 88, 73, 123


rho = 3
epsilon = 0.01

(d_x, d_y) = Lucas_Kanade(start_img[y:y+height,x:x+width],next_img[y:y+height,x:x+width],rho,epsilon,np.zeros(start_img[y:y+height,x:x+width].shape),np.zeros(start_img[y:y+height,x:x+width].shape))
fig, ax = plt.subplots()
d_x_r=cv2.resize(d_x, None, fx=0.3, fy=0.3)
d_y_r=cv2.resize(d_y, None, fx=0.3, fy=0.3)
ax.quiver(-d_x_r,-d_y_r,angles='xy')
plt.title('right_hand (img1,img2)') #ebala titlo edv
plt.gca().set_aspect('equal') 
plt.gca().invert_yaxis()
# plt.savefig('plot.png', dpi=300)
displ_x, displ_y = displ(d_x, d_y, 1)


In [ ]:
Energy = np.zeros(d_x.shape)
for x in range(d_x.shape[0]):
    for y in range(d_y.shape[1]):
        Energy[x,y] = d_x[x,y]**2 + d_y[x,y]**2
plt.title('right_hand_energy (img1,img2)')
plt.imshow(Energy,"gray")

In [ ]:
start_img = cv2.cvtColor(cv2.imread("1.png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
next_img = cv2.cvtColor(cv2.imread("2.png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255

#right hand
#x, y, width, height = 47, 243, 71, 66
#left hand
x, y, width, height = 162, 264, 83, 48
# # face
#x, y, width, height = 138, 88, 73, 123


rho = 3
epsilon = 0.01

(d_x, d_y) = Lucas_Kanade(start_img[y:y+height,x:x+width],next_img[y:y+height,x:x+width],rho,epsilon,np.zeros(start_img[y:y+height,x:x+width].shape),np.zeros(start_img[y:y+height,x:x+width].shape))

fig, ax = plt.subplots()
d_x_r=cv2.resize(d_x, None, fx=0.3, fy=0.3)
d_y_r=cv2.resize(d_y, None, fx=0.3, fy=0.3)
ax.quiver(-d_x_r,-d_y_r,angles='xy')
plt.title('left_hand (img1,img2)')
plt.gca().set_aspect('equal') 
plt.gca().invert_yaxis()
# plt.savefig('plot.png', dpi=300)
displ_x, displ_y = displ(d_x, d_y, 1)


In [ ]:
Energy = np.zeros(d_x.shape)
for x in range(d_x.shape[0]):
    for y in range(d_y.shape[1]):
        Energy[x,y] = d_x[x,y]**2 + d_y[x,y]**2
plt.title('left_hand_energy (img1,img2)')
plt.imshow(Energy,"gray")

In [ ]:
start_img = cv2.cvtColor(cv2.imread("1.png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
next_img = cv2.cvtColor(cv2.imread("2.png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255

#right hand
#x, y, width, height = 47, 243, 71, 66
#left hand
# x, y, width, height = 162, 264, 83, 48
# # face
x, y, width, height = 138, 88, 73, 123


rho = 3
epsilon = 0.01

(d_x, d_y) = Lucas_Kanade(start_img[y:y+height,x:x+width],next_img[y:y+height,x:x+width],rho,epsilon,np.zeros(start_img[y:y+height,x:x+width].shape),np.zeros(start_img[y:y+height,x:x+width].shape))

fig, ax = plt.subplots()
d_x_r=cv2.resize(d_x, None, fx=0.3, fy=0.3)
d_y_r=cv2.resize(d_y, None, fx=0.3, fy=0.3)
ax.quiver(-d_x_r,-d_y_r,angles='xy')
plt.title('face (img1,img2)') #ebala titlo edv
plt.gca().set_aspect('equal') 
plt.gca().invert_yaxis()
# plt.savefig('plot.png', dpi=300)
displ_x, displ_y = displ(d_x, d_y, 1)


In [ ]:
Energy = np.zeros(d_x.shape)
for x in range(d_x.shape[0]):
    for y in range(d_y.shape[1]):
        Energy[x,y] = d_x[x,y]**2 + d_y[x,y]**2
plt.title('face_energy (img1,img2)')
plt.imshow(Energy,"gray")

####1.2.3 Πολυ-Κλιμακωτός Υπολογισμός Οπτικής Ροής

In [ ]:
########################### Tracking ###########################
#right hand
x1, y1, width1, height1 = 47, 243, 71, 66
#left hand
x2,y2,width2,height2 = 162, 264, 83, 48
# face
x3, y3, width3, height3 = 137, 88, 73, 123 

rho = 3
epsilon = 0.01

displ_x1 = 0
displ_y1 = 0
displ_x2 = 0
displ_y2 = 0
displ_x3 = 0
displ_y3 = 0

fig = plt.figure()
ims = []

for i in range(2,67):
    x1 += int(np.round(displ_x1))
    y1 += int(np.round(displ_y1))
    x2 += int(np.round(displ_x2))
    y2 += int(np.round(displ_y2))
    x3 += int(np.round(displ_x3))
    y3 += int(np.round(displ_y3))

    
    #for testing purposes:

    start_img = cv2.cvtColor(cv2.imread(str(i-1)+".png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
    next_img = cv2.cvtColor(cv2.imread(str(i)+".png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
    (d_x1, d_y1) = Lucas_Kanade(start_img[y1:y1+height1,x1:x1+width1],next_img[y1:y1+height1,x1:x1+width1],rho,epsilon,np.zeros(start_img[y1:y1+height1,x1:x1+width1].shape),np.zeros(start_img[y1:y1+height1,x1:x1+width1].shape))
    (d_x2, d_y2) = Lucas_Kanade(start_img[y2:y2+height2,x2:x2+width2],next_img[y2:y2+height2,x2:x2+width2],rho,epsilon,np.zeros(start_img[y2:y2+height2,x2:x2+width2].shape),np.zeros(start_img[y2:y2+height2,x2:x2+width2].shape))
    (d_x3, d_y3) = Lucas_Kanade(start_img[y3:y3+height3,x3:x3+width3],next_img[y3:y3+height3,x3:x3+width3],rho,epsilon,np.zeros(start_img[y3:y3+height3,x3:x3+width3].shape),np.zeros(start_img[y3:y3+height3,x3:x3+width3].shape))
    
    displ_x1, displ_y1 = displ(d_x1, d_y1, .2)
    displ_x2, displ_y2 = displ(d_x2, d_y2, .2)
    displ_x3, displ_y3 = displ(d_x3, d_y3, .2)

    Energy = np.zeros(d_x2.shape)
    for x in range(d_x2.shape[0]):
        for y in range(d_y2.shape[1]):
            Energy[x,y] = d_x2[x,y]**2 + d_y2[x,y]**2
    im = plt.imshow(Energy,"gray")

    # fig, ax = plt.subplots()
    # d_x_r=cv2.resize(d_x, None, fx=0.3, fy=0.3)
    # d_y_r=cv2.resize(d_y, None, fx=0.3, fy=0.3)
    # ax.quiver(-d_x_r,-d_y_r,angles='xy')
    # plt.gca().set_aspect('equal') 
    # plt.gca().invert_yaxis()
    # plt.show()
#     start_img = cv2.cvtColor(cv2.imread(str(i-1)+".png"), cv2.COLOR_BGR2RGB).astype(np.float64)/255
#     gifimage = displayer(start_img.copy(),x1,y1,width1, height1,3)
#     gifimage = displayer(gifimage.copy(),x2,y2,width2, height2,2)
#     gifimage = displayer(gifimage.copy(),x3,y3,width3, height3,1)
#     im = plt.imshow(gifimage)
    ims.append([im])
    print(i)

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=500)
writer = PillowWriter(fps=10)
ani.save("Energy.gif", writer=writer)

plt.show()

In [ ]:
def multiscale_lk(img1,img2,rho,epsilon,d_x0,d_y0,scale):
    for i in range(scale):
        img1 = cv2.pyrDown(img1)
        img2 = cv2.pyrDown(img2)
    for i in range(scale):
        if i == 0:
            (d_x, d_y) = Lucas_Kanade(img1,img2,rho,epsilon,np.zeros(img1.shape),np.zeros(img1.shape))        
        else:
            (d_x, d_y) = Lucas_Kanade(img1,img2,rho,epsilon,d_x,d_y)        
        times = 2 # times the original size
        width1 = int(np.ceil(d_x.shape[1] * times ))
        height1 = int(np.ceil(d_x.shape[0] * times ))
        dim1 = (width1, height1)
        width2 = int(np.ceil(d_y.shape[1] * times ))
        height2 = int(np.ceil(d_y.shape[0] * times ))
        dim2 = (width2, height2)

        d_x = 2*cv2.resize(d_x, dim1)
        d_y = 2*cv2.resize(d_y, dim2)
        
        img1 = cv2.pyrUp(img1)
        img2 = cv2.pyrUp(img2)
    
    return (d_x, d_y)

In [ ]:
########################### Tracking ###########################
#right hand
x1,y1, width1, height1 = 47, 243, 71, 66
#left hand
x2,y2,width2,height2 = 162, 264, 83, 48
# face
x3, y3, width3, height3 = 137, 88, 73, 123 



epsilon = 0.01
rho = 2
threshold = 0.1

displ_x1 = 0
displ_y1 = 0
displ_x2 = 0
displ_y2 = 0
displ_x3 = 0
displ_y3 = 0

fig = plt.figure()
ims = []

for i in range(2,67):
    x1 += int(np.round(displ_x1))
    y1 += int(np.round(displ_y1))
    x2 += int(np.round(displ_x2))
    y2 += int(np.round(displ_y2))
    x3 += int(np.round(displ_x3))
    y3 += int(np.round(displ_y3))

    #for testing purposes:

    start_img = cv2.cvtColor(cv2.imread(str(i-1)+".png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
    next_img = cv2.cvtColor(cv2.imread(str(i)+".png"), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
    (d_x1, d_y1) = multiscale_lk(start_img[y1:y1+height1,x1:x1+width1],next_img[y1:y1+height1,x1:x1+width1],rho,epsilon,np.zeros(start_img[y1:y1+height1,x1:x1+width1].shape),np.zeros(start_img[y1:y1+height1,x1:x1+width1].shape),3)
    (d_x2, d_y2) = multiscale_lk(start_img[y2:y2+height2,x2:x2+width2],next_img[y2:y2+height2,x2:x2+width2],rho,epsilon,np.zeros(start_img[y2:y2+height2,x2:x2+width2].shape),np.zeros(start_img[y2:y2+height2,x2:x2+width2].shape),3)
    (d_x3, d_y3) = multiscale_lk(start_img[y3:y3+height3,x3:x3+width3],next_img[y3:y3+height3,x3:x3+width3],rho,epsilon,np.zeros(start_img[y3:y3+height3,x3:x3+width3].shape),np.zeros(start_img[y3:y3+height3,x3:x3+width3].shape),3)
    
    displ_x1, displ_y1 = displ(d_x1, d_y1, threshold)
    displ_x2, displ_y2 = displ(d_x2, d_y2, threshold)
    displ_x3, displ_y3 = displ(d_x3, d_y3, threshold)

#     start_img = cv2.cvtColor(cv2.imread(str(i-1)+".png"), cv2.COLOR_BGR2RGB).astype(np.float64)/255
#     gifimage = displayer(start_img.copy(),x1,y1,width1, height1,3)
#     gifimage = displayer(gifimage.copy(),x2,y2,width2, height2,2)
#     gifimage = displayer(gifimage.copy(),x3,y3,width3, height3,1)
#     im = plt.imshow(gifimage)
    ims.append([im])
    print("Image: ",i)

    ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=500)
    writer = PillowWriter(fps=20)
    ani.save("Energy.gif", writer=writer)

    plt.show()

##Μέρος 2: Εντοπισμός Χωρο-χρονικών Σημείων Ενδιαφέροντος και Εξαγωγή Χαρακτηριστικών σε Βίντεο Ανθρωπίνων Δράσεων

###2.1 Χωρο-χρονικά Σημεία Ενδιαφέροντος

####2.1.1

In [ ]:
def Harris_detector(vid, s, st, k, threshold, sphere):
    ix, iy, it = np.gradient(vid/255.0)
    
    ns = (ceil(3*s))*2 + 1
    nt = (ceil(3*st))*2 + 1
    
    Gs = cv2.getGaussianKernel(ns, s) @ cv2.getGaussianKernel(ns, s).T
    Gt = cv2.getGaussianKernel(nt, st)
    
    """
                                     [L_x^2, L_xy  , L_xt ] 
    M = g(x,y,t;sσ, sτ) (syneliksi)  [L_yx , L_y ^2, L_yt ]
                                     [L_tx , L_ty  , L_t^2]
    """
    
    Lxx = cv2.filter2D(cv2.filter2D(ix*ix, -1, Gs), -1, Gt)
    Lyy = cv2.filter2D(cv2.filter2D(iy*iy, -1, Gs), -1, Gt)
    Ltt = cv2.filter2D(cv2.filter2D(it*it, -1, Gs), -1, Gt)
    Lxy = cv2.filter2D(cv2.filter2D(ix*iy, -1, Gs), -1, Gt)
    Lxt = cv2.filter2D(cv2.filter2D(ix*it, -1, Gs), -1, Gt)
    Lyt = cv2.filter2D(cv2.filter2D(iy*it, -1, Gs), -1, Gt)

    M_trace = Lxx + Lyy + Ltt
    
    det = Lxx*Lyy*Ltt + 2*Lxy*Lxt*Lyt - Lxx*Lyt**2 - Lyy*Lxt**2 - Ltt*Lxy**2
    
    R = abs(det - k*M_trace**3)
    
    R_max = R.max()
    B_sq = disk_strel(ns)
    Cond1 = (R == cv2.dilate(R,B_sq))
    Cond2 = (R > threshold*R_max)
    cond = Cond1&Cond2
    result = np.where(cond == True)
    listOfCoordinates= list(zip(result[0], result[1], result[2]))
    arr = []
#     fig = plt.figure()
#     ims = []
#     for i in range(vid.shape[2]):
#         im = plt.imshow(R[:,:,i],"gray_r")
#         ims.append([im])
#     ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=500)
#     writer = PillowWriter(fps=20)
#     ani.save("Harris.gif", writer=writer)
    for coordinate in listOfCoordinates:
        arr.append([coordinate[1], coordinate[0],coordinate[2], 3])
    return np.array(arr)

In [ ]:
#path argument must be in form 'folder/.../subfolder/'
def display_video(video, extra_points, path):
    utils.show_detection(video, extra_points, save_path = path)

    num_frames = video.shape[2]

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    org_img = cv2.imread(path + 'frame0.png')
    out = cv2.VideoWriter(path+'video.mp4', fourcc, 12.0, (org_img.shape[1], org_img.shape[0]))


    for i in range(1, num_frames):
        org_img = cv2.imread(path+'frame' + str(i)+'.png')
        out.write(org_img)

    out.release()
    cv2.destroyAllWindows() 
    print("Created the video")

    return

####2.1.2

In [ ]:
from numpy import unravel_index

def oddGabor(taf, omega,t):
    mycos = np.cos(2*np.pi*t*omega)
    myexp = np.exp(-t**2 / (2 * taf**2))
    return  mycos*myexp/sum(mycos*myexp)

def evGabor(taf, omega,t):
    mysin = np.sin(2*np.pi*t*omega)
    myexp = np.exp(-t**2 / (2 * taf**2))
    return  mysin*myexp/sum(mysin*myexp)
         

def Gabor(myvideo,taf,sigma):
    imgev = np.zeros(myvideo.shape)
    imgod = np.zeros(myvideo.shape)
    omega = 4/taf
    for i in range(myvideo.shape[2]):
        t = np.linspace(start = -2*taf, stop = 2*taf, num = int(np.round(4*taf+1)))
        hod = oddGabor(taf,omega,t)
        hev = evGabor(taf,omega,t)
        Img = myvideo[:,:,i]
        imgev[:,:,i] = Gauss(Img,sigma)
        imgod[:,:,i] = Gauss(Img,sigma)
    H = np.zeros(myvideo.shape)    
    for x in range(myvideo.shape[0]):
        for y in range(myvideo.shape[1]):
            H[x,y] = (np.convolve(imgev[x,y],hev,mode = 'same')  **2 + np.convolve(imgod[x,y],hod,mode = 'same') ** 2)
#     fig = plt.figure()
#     ims = []
#     for i in range(myvideo.shape[2]):
#         im = plt.imshow(H[:,:,i],"gray_r")
#         ims.append([im])
#     ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=500)
#     writer = PillowWriter(fps=20)
#     ani.save("Harris.gif", writer=writer)
    
    some = []
    for i in range(myvideo.shape[2]):
        value = 2
        x1,y1 = unravel_index(H[:,:,i].argmax(), H[:,:,i].shape)
        some.append([y1,x1,i,3])
        x2 = x1
        y2 = y1
        while(mydist(x1,y1,x2,y2)):
    #         print(mydist(x1,y1,x2,y2),x1,y1,x2,y2)
            second = np.where(H[:,:,i] == np.partition(H[:,:,i].flatten(), -value)[-value])
            x2,y2 = (second[0][0],second[1][0])
            value += 1
        some.append([y2,x2,i,3])
        x3 = x1
        y3 = y1
        while(mydist(x1,y1,x3,y3) or mydist(x2,y2,x3,y3)):
            a = np.where(H[:,:,i] == np.partition(H[:,:,i].flatten(), -value)[-value])
            x3,y3 = (a[0][0],a[1][0])
            value += 1
        some.append([y3,x3,i,3])
        x4 = x1
        y4 = y1
        while(mydist(x1,y1,x4,y4) or mydist(x2,y2,x4,y4) and mydist(x3,y3,x4,y4)):
            b = np.where(H[:,:,i] == np.partition(H[:,:,i].flatten(), -value)[-value])
            x4,y4 = (b[0][0],b[1][0])
            value += 1
        some.append([y4,x4,i,3])
    return some

def mydist(x1,y1,x2,y2):
    return (x1-x2)** 2 + (y1-y2) ** 2 <= 16

####2.1.3

In [ ]:
video = utils.read_video("Videos/person04_walking_d1_uncomp.avi", 100, 0)
arr = Harris_detector(video, 2,4,0.005,0.0005,2)
display_video(video, arr, "harris/")

In [ ]:
nframes = 200
myvideo = utils.read_video("Videos/person06_running_d2_uncomp.avi",nframes,0)
arr = Gabor(myvideo, 1.5, 4)
display_video(myvideo,arr,path = "gabor/")

###2.2 Χωρο-χρονικοί Ιστογραφικοί Περιγραφητές

In [ ]:
def Calculate_Descriptors(video, points_of_interest, HOG, nbins, grid_size):
    #print("Calculating Descriptors...")
    if HOG:
        Gx, Gy, Gt = np.gradient(video)
    else:
        dx, dy = [0]*(video.shape[2]-1),[0]*(video.shape[2]-1)
        for i in range(video.shape[2]-1):
            dx[i], dy[i] = multiscale_lk(video[:,:, i],
                                         video[:,:, i+1],
                                         rho,epsilon,
                                         np.zeros(video[:,:, i].shape),
                                         np.zeros(video[:,:, i].shape),3)
    
    desc_window_size = int(grid_size / 2)
    descriptors = []
    for point in points_of_interest:
        x,y,t,scale = point
        if (t == video.shape[2] - 1):
            continue
        
        
        if x-desc_window_size < 0:
            x_down_limit = 0
        else:
            x_down_limit  = x-desc_window_size
        
        
        if x + desc_window_size >= video.shape[1]:
            x_up_limit = video.shape[1] - 1
        else:
            x_up_limit = x + desc_window_size
        
        
        
        if y-desc_window_size < 0:
            y_down_limit = 0
        else:
            y_down_limit = y-desc_window_size
        
        
        if y + desc_window_size >= video.shape[0]:
            y_up_limit = video.shape[0] - 1
        else:
            y_up_limit = y + desc_window_size
        
        x_down_limit = int(round(x_down_limit))
        x_up_limit = int(round(x_up_limit))
        y_down_limit = int(round(y_down_limit))
        y_up_limit = int(round(y_up_limit))
        t = int(round(t))
        
        if HOG:
            gx = Gx[y_down_limit:y_up_limit, x_down_limit:x_up_limit, t]
            gy = Gy[y_down_limit:y_up_limit, x_down_limit:x_up_limit, t]
        else:
            gx = dx[t][y_down_limit:y_up_limit, x_down_limit:x_up_limit]
            gy = dy[t][y_down_limit:y_up_limit, x_down_limit:x_up_limit]
        descriptor = utils.orientation_histogram(gx, gy, nbins, np.array([grid_size,grid_size]))
        if not all(descriptor == 0):
            descriptors.append(descriptor)
        
        
    return descriptors

###2.3: Κατασκευή Bag of Visual Words και χρήση Support Vector Machines για την ταξινόμηση δράσεων

####2.3.1

In [ ]:
def split_train_test(path, testing_size=0.2):
    video_list = []
    video_names = [f for f in listdir(path) if isfile(join(path, f))]
    for name in video_names:
        video_list.append([name, utils.read_video(path+name, 300, 0)])
    train, test = train_test_split(video_list, test_size=testing_size) 
    return (train, test)

In [ ]:
train_set, test_set = split_train_test("Videos/")
print(len(train_set))
print(len(test_set))

In [ ]:
#################################### Descriptors using Harris detector ####################################

harris_hog_desc_train = []
harris_hog_desc_test = []

harris_hof_desc_train = []
harris_hof_desc_test = []


for video in train_set:
    arr = Harris_detector(video[1], 2,0.7,0.005,0.0005,2)
    harris_hog_desc_train.append(Calculate_Descriptors(video[1], arr, True,8, 4*2))
    harris_hof_desc_train.append(Calculate_Descriptors(video[1], arr, False,8, 4*2))
print("Done training set!")
for video in test_set:
    arr = Harris_detector(video[1], 2,0.7,0.005,0.0005,2)
    harris_hog_desc_test.append(Calculate_Descriptors(video[1], arr, True, 8, 4*2))
    harris_hof_desc_test.append(Calculate_Descriptors(video[1], arr, False, 8, 4*2))
print("Done with the Harris Descriptors!")

######## Using the Concatenation of both ########
harris_hog_hof_desc_train = []
harris_hog_hof_desc_test = []
for i in range(len(harris_hog_desc_train)):
    harris_hog_hof_desc_train.append(harris_hog_desc_train[i] + harris_hof_desc_train[i])
for i in range(len(harris_hog_desc_test)):
    harris_hog_hof_desc_test.append(harris_hog_desc_test[i] + harris_hof_desc_test[i])
print("Done with the Harris Descriptors!")

#################################### Descriptors using Gabor detector ####################################

gabor_hog_desc_train = []
gabor_hog_desc_test = []

gabor_hof_desc_train = []
gabor_hof_desc_test = []


for video in train_set:
    arr = Gabor(video[1], 1.5, 4)
    gabor_hog_desc_train.append(Calculate_Descriptors(video[1], arr, True,8, 4*2))
    gabor_hof_desc_train.append(Calculate_Descriptors(video[1], arr, False,8, 4*2))
print("Done training set!")
for video in test_set:
    arr = Gabor(video[1], 1.5, 4)
    gabor_hog_desc_test.append(Calculate_Descriptors(video[1], arr, True, 8, 4*2))
    gabor_hof_desc_test.append(Calculate_Descriptors(video[1], arr, False, 8, 4*2))
print("Done with the Gabor Descriptors!")

######## Using the Concatenation of both ########
gabor_hog_hof_desc_train = []
gabor_hog_hof_desc_test = []

for i in range(len(gabor_hog_desc_train)):
    gabor_hog_hof_desc_train.append(gabor_hog_desc_train[i] + gabor_hof_desc_train[i])
for i in range(len(gabor_hog_desc_test)):
    gabor_hog_hof_desc_test.append(gabor_hog_desc_test[i] + gabor_hof_desc_test[i])

####2.3.2

In [ ]:
#1
for i in range(len(harris_hog_desc_train)):
    harris_hog_desc_train[i] = np.array(harris_hog_desc_train[i])
harris_hog_desc_train = np.array(harris_hog_desc_train)
for i in range(len(harris_hog_desc_test)):
    harris_hog_desc_test[i] = np.array(harris_hog_desc_test[i])
harris_hog_desc_test = np.array(harris_hog_desc_test)
#2
for i in range(len(harris_hof_desc_train)):
    harris_hof_desc_train[i] = np.array(harris_hof_desc_train[i])
harris_hof_desc_train = np.array(harris_hof_desc_train)
for i in range(len(harris_hof_desc_test)):
    harris_hof_desc_test[i] = np.array(harris_hof_desc_test[i])
harris_hof_desc_test = np.array(harris_hof_desc_test)
#3
for i in range(len(harris_hog_hof_desc_train)):
    harris_hog_hof_desc_train[i] = np.array(harris_hog_hof_desc_train[i])
harris_hog_hof_desc_train = np.array(harris_hog_hof_desc_train)
for i in range(len(harris_hog_hof_desc_test)):
    harris_hog_hof_desc_test[i] = np.array(harris_hog_hof_desc_test[i])
harris_hog_hof_desc_test = np.array(harris_hog_hof_desc_test)
#4
for i in range(len(gabor_hog_desc_train)):
    gabor_hog_desc_train[i] = np.array(gabor_hog_desc_train[i])
gabor_hog_desc_train = np.array(gabor_hog_desc_train)
for i in range(len(gabor_hog_desc_test)):
    gabor_hog_desc_test[i] = np.array(gabor_hog_desc_test[i])
gabor_hog_desc_test = np.array(gabor_hog_desc_test)
#5
for i in range(len(gabor_hof_desc_train)):
    gabor_hof_desc_train[i] = np.array(gabor_hof_desc_train[i])
gabor_hof_desc_train = np.array(gabor_hof_desc_train)
for i in range(len(gabor_hof_desc_test)):
    gabor_hof_desc_test[i] = np.array(gabor_hof_desc_test[i])
gabor_hof_desc_test = np.array(gabor_hof_desc_test)
#6
for i in range(len(gabor_hog_hof_desc_train)):
    gabor_hog_hof_desc_train[i] = np.array(gabor_hog_hof_desc_train[i])
gabor_hog_hof_desc_train = np.array(gabor_hog_hof_desc_train)
for i in range(len(gabor_hog_hof_desc_test)):
    gabor_hog_hof_desc_test[i] = np.array(gabor_hog_hof_desc_test[i])
gabor_hog_hof_desc_test = np.array(gabor_hog_hof_desc_test)

In [ ]:
bow_train_Gabor_HOG, bow_test_Gabor_HOG = utils.bag_of_words(gabor_hog_desc_train, gabor_hog_desc_test, 48)
bow_train_Gabor_HOF, bow_test_Gabor_HOF = utils.bag_of_words(gabor_hof_desc_train, gabor_hof_desc_test, 48)
bow_train_Gabor_HOG_HOF, bow_test_Gabor_HOG_HOF = utils.bag_of_words(gabor_hog_hof_desc_train, gabor_hog_hof_desc_test, 48)

####2.3.3

In [ ]:
def make_name_right(name):
    return name.split('_')[1]

In [ ]:
train_labels = []
test_labels = []
for video in train_set:
    train_labels.append(make_name_right(video[0]))
for video in test_set:
    test_labels.append(make_name_right(video[0]))
    
    
accuracy1, prediction1 = utils.svm_train_test(bow_train_Harris_HOG, train_labels, bow_test_Harris_HOG, test_labels)
accuracy2, prediction2 = utils.svm_train_test(bow_train_Harris_HOF, train_labels, bow_test_Harris_HOF, test_labels)
accuracy3, prediction3 = utils.svm_train_test(bow_train_Harris_HOG_HOF, train_labels, bow_test_Harris_HOG_HOF, test_labels)
accuracy4, prediction4 = utils.svm_train_test(bow_train_Gabor_HOG, train_labels, bow_test_Gabor_HOG, test_labels)
accuracy5, prediction5 = utils.svm_train_test(bow_train_Gabor_HOF, train_labels, bow_test_Gabor_HOF, test_labels)
accuracy6, prediction6 = utils.svm_train_test(bow_train_Gabor_HOG_HOF, train_labels, bow_test_Gabor_HOG_HOF, test_labels)

In [ ]:
print("Harris_HOG",accuracy1)
print("Harris_HOF",accuracy2)
print("Harris_HOG_HOF",accuracy3)
print("Gabor_HOG",accuracy4)
print("Gabor_HOF",accuracy5)
print("Gabor_HOG_HOF",accuracy6)